In [1]:
import sys
sys.path.append("C:\ZSOMBI\OneDrive\PoPy")
sys.path.append("/Users/zsombi/OneDrive/PoPy")

import pandas as pd
from tqdm import tqdm
import os

from popy.io_tools import *
from popy.dim_reduction import *
from popy.behavior_data_tools import *
from popy.neural_data_tools import time_normalize_session, scale_neural_data, remove_low_fr_neurons, remove_trunctuated_neurons
from popy.decoders import *
from popy.plotting_tools import *
import popy.config as cfg

Using OFFICE path
Inserm drive is not accessible


In [2]:
project_folder = cfg.PROJECT_PATH_LOCAL

# sessions info
sessions = pd.read_pickle(os.path.join(project_folder, 'data', 'recordings_summary.pickle'))
out_path = os.path.join(project_folder, 'data', 'processed', 'rates')

neural_data_example = xr.open_dataarray(os.path.join(project_folder, 'data', 'processed', 'rates', 'neural_data_ka_010720.nc'))
# Load data
# load results

session_data = {}

for i, row in sessions.iterrows():
    if row.monkey != 'ka':
        continue
    if not row.behav_valid:
        continue
    if row.interrupted_trials != 0:
        continue

    monkey, session = row.monkey, row.session
    try:
        # load behavioral data
        behav_data = get_behavior(monkey, session)
        behav_data = add_value_function(behav_data)
        behav_data = add_switch_info(behav_data)
        behav_data = add_phase_info(behav_data, exploration_limit=6, transition_limit=1)
        behav_data['value_function'] = behav_data['value_function'].shift(-1)
        behav_data['switch'] = behav_data['switch'].shift(-1)
        behav_data['phase'] = behav_data['phase'].shift(-1)
        behav_data = add_RPE(behav_data)
        behav_data = behav_data.dropna()
        behav_data['value_function_digit'] = np.digitize(behav_data['value_function'], [0, .25, .5, .75, 1]) - 1
        behav_data['RPE_digit'] = np.digitize(behav_data['RPE'], [0, .25, .5, .75, 1]) - 1
        #behav_data = drop_time_fields(behav_data)  # remove time fields
        behav_data = behav_data.drop(['block_id', 'best_target'], axis=1)  # drop block_id and best_target

        # Load neural data
        out_path = os.path.join(cfg.PROJECT_PATH_LOCAL, 'data', 'processed', 'rates')
        floc = os.path.join(out_path , f'neural_data_{monkey}_{session}.nc')
        neural_data = xr.open_dataarray(floc)  #load already preprocessed neual data

        # match neural and behavioral data
        neural_trials = neural_data.trial_id.values
        behav_trials = behav_data.trial_id.values
        shared_trials = np.intersect1d(neural_trials, behav_trials)
        session_data[session] = {'MCC': {'neural_data': None, 
                                         'behav_data': None}, 
                                 'LPFC': {'neural_data': None, 
                                          'behav_data': None}}
        for area in ['MCC', 'LPFC']:
            neural_data_area = neural_data[neural_data.area == area]  # select only PFC
            session_data[session][area]['neural_data'] = neural_data_area[:, neural_data_area.trial_id.isin(shared_trials)]
            session_data[session][area]['behav_data'] = behav_data[behav_data.trial_id.isin(shared_trials)]

    except:
        print(f'Session not found: {monkey}_{session}_{area}')
        continue

adding history of feedback column, but if the data is pooled across sessions, make sure to add the history before pooling data to keep the order of time!
adding history of feedback column, but if the data is pooled across sessions, make sure to add the history before pooling data to keep the order of time!
adding history of feedback column, but if the data is pooled across sessions, make sure to add the history before pooling data to keep the order of time!
Session not found: ka_190620_LPFC
adding history of feedback column, but if the data is pooled across sessions, make sure to add the history before pooling data to keep the order of time!
adding history of feedback column, but if the data is pooled across sessions, make sure to add the history before pooling data to keep the order of time!
Session not found: ka_090720_LPFC
adding history of feedback column, but if the data is pooled across sessions, make sure to add the history before pooling data to keep the order of time!
adding h

In [3]:
## Generate PCA
monkey = 'ka'
for session, data_all in session_data.items():
    for area, data in data_all.items():
        print(session, area)
        neural_data_area = data['neural_data']
        if neural_data_area.shape[0] <= 4:
            continue
        behav_data = data['behav_data']
        #modes = ['full', 'mean', 'full_cut', 'mean_cut', 'behav']
        pca_of_interest = 'full'
        condition_of_interest = 'target'

        # fit PCA
        pca = fit_model(behav_data, neural_data_area, mode=pca_of_interest,
                        behav_condition=condition_of_interest)

        PCA_data = neural_data_area.copy(data=pca.transform(neural_data_area.T).T)

        # prepare data for plotting
        # df to collect PCA trajectories for plottingresults per condition
        T_all = pd.DataFrame(columns=['feedback', 'RPE_digit', 'value_function_digit', 'switch', 'target'],
                                index=[area])

        n_components = 4  # number of components to use for plotting
        for condition in T_all.columns:
            T_full, labels = build_dataset(PCA_data, behav_data, 
                                        starts_on='fb-1', ends_on='fb+2',
                                        target_name=condition)

            T_current = {}
            # get mean trial activity per condition
            for label in np.unique(labels):
                T_temp = np.mean(T_full[labels == label], axis=0)  # mean activity per condition
                #T_temp = T_temp - T_mean  # subtract full mean activity
                T_current[label] = T_temp[:n_components, :]  # save only the first n_components
            T_all[condition][area] = T_current

        # plot
        title = f'Representation of conditions in PC space, {monkey} {session} {area} {pca_of_interest}'
        savedir = os.path.join(project_folder, 'figs', 'pca', 'all', f'{monkey}_{session}_{area}_pca_full.png')
        show_PCA_trajectories(neural_data, T_all, pca.explained_variance_ratio_, n_components=n_components, 
        title=title, savedir=savedir, show=False)

050620 MCC
050620 LPFC
100620 MCC
100620 LPFC
010720 MCC
010720 LPFC
150720 MCC
150720 LPFC
230720 MCC
230720 LPFC
270720 MCC
270720 LPFC
300720 MCC
300720 LPFC
100820 MCC
100820 LPFC
120820 MCC
120820 LPFC
240820 MCC
240820 LPFC
310820 MCC
310820 LPFC
100920 MCC
100920 LPFC
011020 MCC
011020 LPFC
221020 MCC
221020 LPFC
281020 MCC
281020 LPFC
150221 MCC
150221 LPFC
040321 MCC
040321 LPFC
110321 MCC
110321 LPFC
180321 MCC
180321 LPFC
120821 MCC
120821 LPFC
190821 MCC
190821 LPFC
290921 MCC
290921 LPFC
030322 MCC
030322 LPFC
070322 MCC
070322 LPFC
210322 MCC
210322 LPFC
280322 MCC
280322 LPFC
050422 MCC
050422 LPFC
130422 MCC
130422 LPFC
200422 MCC
200422 LPFC
250422 MCC
250422 LPFC
030522 MCC
030522 LPFC
020622 MCC
020622 LPFC
080622 MCC
080622 LPFC
130622 MCC
130622 LPFC
200622 MCC
200622 LPFC
280622 MCC
280622 LPFC
300622 MCC
300622 LPFC
070722 MCC
070722 LPFC
080822 MCC
080822 LPFC
300822 MCC
300822 LPFC
060922 MCC
060922 LPFC


In [4]:
## Merge plots to one big pdf
# merge pngs in the location to one main pdf
import os
from PIL import Image
#import img2pdf
from pypdf import PdfMerger


def merge_pdfs(path, output_name):
    pdfs = []
    for file in os.listdir(path):
        if file.endswith("_pca_full.pdf"):
            pdfs.append(os.path.join(path, file))

    merger = PdfMerger()

    # sort by name
    pdfs.sort()

    # change the order from 1,2,3,4... to 2,1,4,3...
    pdfs = [item for sublist in zip(pdfs[1::2], pdfs[::2]) for item in sublist]
    print(pdfs)

    for pdf in pdfs:
        merger.append(pdf)

    merger.write(output_name)
    merger.close()

path = os.path.join(project_folder, 'figs', 'pca', 'all')
output_name = os.path.join(project_folder, 'figs', 'pca', 'summary.pdf')
merge_pdfs(path, output_name)

print("summary.pdf created at " + output_name)

[]
summary.pdf created at C:\ZSOMBI\OneDrive\PoPy\figs\pca\summary.pdf
